In [16]:
import os

PATH = '/home/stadlerm/Development/Projects/DreamChallenge/RA2_alpine_lads'

os.chdir(PATH)

In [2]:
from utils.config import Config

config = Config()

In [264]:
import os
import pandas as pd

training_csv = os.path.join(config.train_location, 'training.csv')

training_data = pd.read_csv(training_csv)

In [299]:
# Prepare outcome vector


column_types = [
    ['hands_narrowing', '.*H_.*_J.*'], 
    ['hands_erosion', '.*H_.*_E.*'], 
    ['feet_narrowing', '.*F_.*_J.*'],
    ['feet_erosion', '.*F_.*_E.*']
]

for column_type in column_types:
    file_post_fix = column_type[0]
    column_reg = column_type[1]
    
    p = re.compile(column_reg)

    column_idx = np.where([p.match(column) for column in training_data.columns])[0]
    columns = training_data.columns[column_idx]
    
    values = training_data[columns]
    vals, counts = np.unique(values, return_counts = True)
    
    y = np.zeros((training_data.shape[0], vals.size))
    
    for idx, val in enumerate(vals):
        r_idx = np.where(values == val)[0]
        
        y[r_idx, idx] = 1
        
    y_t = y[:, 1]

    for idx in range(1, vals.size):
        y_t = np.logical_and(y_t, y[:, idx])
    
    print(np.count_nonzero(y_t))

6
2
1
0


In [296]:
import functools

y_t = y[:, 0]

for idx in range(1, 5):
    y_t = np.logical_and(y_t, y[:, idx])
    
print(np.count_nonzero(y_t))

6


In [200]:
# Regex for hand narrowing, erosion, and feet narrowing columns
column_reg = ['.*H_.*_J.*', '.*H_.*_E.*', '.*F_.*_J.*']

for idx, reg in enumerate(column_reg):
    p = re.compile(reg)

    column_idx = np.where([p.match(column) for column in training_data.columns])[0]
    columns = training_data.columns[column_idx]
    
    add_y_classes(training_data[columns].to_numpy(), idx)

# Process feet erosion differently because it goes from 0-10, and higher values are extremely rare (8-10)
p = re.compile('.*F_.*_E.*')
column_idx = np.where([p.match(column) for column in training_data.columns])[0]
columns = training_data.columns[column_idx]

values = np.round(training_data[columns] / 2)
# Very few have 5, so add them to 4
values[values == 5] = 4
add_y_classes(values, 3)

In [230]:
print(np.unique(y[:, 0], return_counts = True))
print(np.unique(y[:, 1], return_counts = True))
print(np.unique(y[:, 2], return_counts = True))
print(np.unique(y[:, 3], return_counts = True))

(array([0., 1., 2., 3., 4.]), array([163,  22,  65,  37,  81]))
(array([0., 1., 2., 3., 4., 5.]), array([145, 107,  45,  53,   4,  14]))
(array([0., 1., 2., 3., 4.]), array([257,   7,  47,  28,  29]))
(array([0., 1., 2., 3., 4.]), array([268,  66,  26,   4,   4]))


In [231]:
def create_joint_random_variable(x, y):
    N = x.shape[0]

    # Alphabet of x & y
    x_abc = np.unique(x)
    y_abc = np.unique(y)

    # Joint Random Variable
    x_y = np.zeros(N, dtype = np.int32)

    # Hold current combination
    n = 0
    for x_letter in x_abc:
        for y_letter in y_abc:
            # Check entries where x & y are equal to the current letter
            x_y_idx = np.logical_and(x == x_letter, y == y_letter)

            # Skip non existent combinations
            if (np.count_nonzero(x_y_idx) > 0):
                x_y[x_y_idx] = n

                # After a combination has been used, increase counter to assign a new value to new combinations
                n += 1

    return x_y

In [241]:
print(y)

[[2. 1. 1. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [3. 0. 0. 0.]
 [4. 1. 0. 0.]
 [2. 1. 3. 0.]]


In [238]:
joint_y = y[:, 0]

for y_p in range(3):
        joint_y = create_joint_random_variable(joint_y, y[:, y_p+1])

In [245]:
joint_y = create_joint_random_variable(create_joint_random_variable(create_joint_random_variable(y[:, 0], y[:, 1]), y[:, 2]), y[:, 3])

In [247]:
np.unique(joint_y, return_counts = True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106], dtype=int32),
 array([64, 12,  3,  7,  2,  3,  1,  2,  1,  1, 42,  7,  3,  3,  1,  1,  8,
         1,  1,  5,  3,  1,  1,  1,  1,  1,  1,  2,  1,  2,  1,  1,  1, 19,
         1,  4,  1, 12,  4,  1,  1,  3,  1,  1,  1,  1,  3,  1,  1,  2,  2,
         4,  1,  1, 14,  1,  2,  4,  3,  3,  1,  2,  4,  1,  1,  1,  6,  1,
         5,  1,  1,  3,  1,  1,  4,  1,  1,  1,  1,  1,  2,  1, 10,  1,  2

In [254]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(np.arange(training_data.shape[0]), test_size = 0.25, stratify = y[:, 2])

In [300]:
print(np.unique(y[train_idx, 0], return_counts = True))
print(np.unique(y[train_idx, 1], return_counts = True))
print(np.unique(y[train_idx, 2], return_counts = True))
print(np.unique(y[train_idx, 3], return_counts = True))

(array([0., 1.]), array([  1, 275]))
(array([0., 1.]), array([190,  86]))
(array([0., 1.]), array([228,  48]))
(array([0., 1.]), array([253,  23]))


In [301]:
print(np.unique(y[test_idx, 0], return_counts = True))
print(np.unique(y[test_idx, 1], return_counts = True))
print(np.unique(y[test_idx, 2], return_counts = True))
print(np.unique(y[test_idx, 3], return_counts = True))

(array([1.]), array([92]))
(array([0., 1.]), array([66, 26]))
(array([0., 1.]), array([69, 23]))
(array([0., 1.]), array([84,  8]))


In [257]:
train_patient_id = training_data.iloc[train_idx]['Patient_ID'].to_numpy()
test_patient_id = training_data.iloc[test_idx]['Patient_ID'].to_numpy()

In [258]:
print(train_patient_id)

['UAB227' 'UAB345' 'UAB343' 'UAB614' 'UAB040' 'UAB189' 'UAB223' 'UAB489'
 'UAB046' 'UAB052' 'UAB075' 'UAB232' 'UAB128' 'UAB394' 'UAB366' 'UAB389'
 'UAB319' 'UAB661' 'UAB196' 'UAB151' 'UAB379' 'UAB522' 'UAB039' 'UAB323'
 'UAB268' 'UAB224' 'UAB503' 'UAB204' 'UAB300' 'UAB239' 'UAB126' 'UAB270'
 'UAB286' 'UAB384' 'UAB463' 'UAB567' 'UAB013' 'UAB219' 'UAB090' 'UAB080'
 'UAB033' 'UAB023' 'UAB392' 'UAB630' 'UAB139' 'UAB054' 'UAB557' 'UAB086'
 'UAB551' 'UAB260' 'UAB233' 'UAB717' 'UAB143' 'UAB491' 'UAB539' 'UAB081'
 'UAB002' 'UAB105' 'UAB563' 'UAB436' 'UAB006' 'UAB597' 'UAB238' 'UAB045'
 'UAB131' 'UAB005' 'UAB207' 'UAB041' 'UAB264' 'UAB009' 'UAB210' 'UAB573'
 'UAB440' 'UAB596' 'UAB087' 'UAB487' 'UAB317' 'UAB109' 'UAB494' 'UAB372'
 'UAB171' 'UAB427' 'UAB093' 'UAB406' 'UAB411' 'UAB244' 'UAB495' 'UAB140'
 'UAB458' 'UAB065' 'UAB018' 'UAB180' 'UAB617' 'UAB422' 'UAB360' 'UAB191'
 'UAB216' 'UAB267' 'UAB701' 'UAB031' 'UAB085' 'UAB129' 'UAB357' 'UAB700'
 'UAB177' 'UAB519' 'UAB124' 'UAB335' 'UAB694' 'UAB4

In [259]:
print(test_patient_id)

['UAB212' 'UAB517' 'UAB067' 'UAB183' 'UAB654' 'UAB500' 'UAB135' 'UAB078'
 'UAB161' 'UAB246' 'UAB504' 'UAB310' 'UAB702' 'UAB679' 'UAB582' 'UAB030'
 'UAB056' 'UAB142' 'UAB064' 'UAB595' 'UAB665' 'UAB123' 'UAB104' 'UAB581'
 'UAB062' 'UAB098' 'UAB482' 'UAB502' 'UAB648' 'UAB099' 'UAB560' 'UAB720'
 'UAB348' 'UAB653' 'UAB472' 'UAB178' 'UAB333' 'UAB307' 'UAB301' 'UAB706'
 'UAB683' 'UAB625' 'UAB159' 'UAB628' 'UAB042' 'UAB036' 'UAB160' 'UAB205'
 'UAB277' 'UAB145' 'UAB115' 'UAB441' 'UAB352' 'UAB490' 'UAB569' 'UAB645'
 'UAB271' 'UAB280' 'UAB537' 'UAB688' 'UAB106' 'UAB066' 'UAB061' 'UAB051'
 'UAB538' 'UAB347' 'UAB182' 'UAB130' 'UAB666' 'UAB057' 'UAB168' 'UAB198'
 'UAB667' 'UAB443' 'UAB113' 'UAB150' 'UAB697' 'UAB602' 'UAB334' 'UAB217'
 'UAB228' 'UAB501' 'UAB127' 'UAB393' 'UAB144' 'UAB162' 'UAB122' 'UAB297'
 'UAB669' 'UAB337' 'UAB580' 'UAB050']


In [260]:
hands_dataframe = pd.read_csv('./data/hand_joint_data.csv')
feet_dataframe = pd.read_csv('./data/feet_joint_data.csv')

In [261]:
hand_idx = [image_name.split('-')[0] in test_patient_id for image_name in hands_dataframe['image_name']]
feet_idx = [image_name.split('-')[0] in test_patient_id for image_name in feet_dataframe['image_name']]

In [262]:
hands_dataframe.iloc[np.where(np.logical_not(hand_idx))[0]].to_csv('./data/hand_joint_data_train.csv')
feet_dataframe.iloc[np.where(np.logical_not(feet_idx))[0]].to_csv('./data/feet_joint_data_train.csv')

hands_dataframe.iloc[np.where(hand_idx)[0]].to_csv('./data/hand_joint_data_test.csv')
feet_dataframe.iloc[np.where(feet_idx)[0]].to_csv('./data/feet_joint_data_test.csv')

In [302]:
feet_data = pd.read_csv('./data/feet_joint_data_train.csv')
feet_data_val = pd.read_csv('./data/feet_joint_data_test.csv')

In [304]:
np.unique(feet_data['narrowing_0'], return_counts = True)

(array([0., 1., 2., 3., 4.]), array([3016,    5,  143,   76,   72]))

In [305]:
np.unique(feet_data_val['narrowing_0'], return_counts = True)

(array([0., 1., 2., 3., 4.]), array([990,   2,  54,  30,  28]))

In [309]:
x = np.count_nonzero(feet_data_val['narrowing_0'] == 0)

In [310]:
print(x / feet_data_val.shape[0])

0.8967391304347826
